# ModelIO

> 해당 자료 참고 링크 : [langchain v0.1 기준 Docs 페이지링크 : 현재 유지보수 지원하지 않는 버전](https://python.langchain.com/v0.1/docs/modules/)

- prompt
- chat models
- llms


## FewshotPromptTemplate


In [28]:

# 기본 방식
t = PromptTemplate(
    template="{country} 수도는?",
    input_variables = ["country"]
    )

t = PromptTemplate.from_template(
    "{country} 수도는?",
    )
t.format(country = "미국")


'미국 수도는?'

In [29]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate 
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotPromptTemplate # 어떻게 대답해야하는지 LM에게 알려주기 위한 예제를 알려주는 것

model="gpt-4o-mini"
chat = ChatOpenAI(
    model_name=model,
    temperature=0.1,
    streaming = True, # LLM response가 생성되는대로 볼 수 있도록 함 /전체 응답이 끝나기 전에 한글자라도 나오면 볼수있음
    callbacks = [StreamingStdOutCallbackHandler(),] # 응답을 콘솔로 바로 print 해줌 / callback : 다양한 이벤트들을 감지할 수 있음(llm 시작 종료 등)
)
# 대화기록같은걸 DB에서 가져와서 예제로 넣어주면 형식화하여 답변을 더 잘할 것
# 보고서 형식/문법이라던가 대본 등등

# 예제가 없는 경우
# chat.predict("김치찌개 설명해줘")

examples = [{
    "question":"된장찌개 설명해줘",
    "answer" : """
    나라 : 한국
    주요 재료 : 된장, 두부, 호박
    비슷한 음식 : 청국장
    """
    },{
    "question":"김치찌개",
    "answer" : """
    나라 : 한국
    주요 재료 : 김치, 돼지고기, 양파 
    비슷한 음식 : 고추장찌개
    """
    },
    {
    "question":"훠거 설명해줘",
    "answer" : """
    나라 : 중국
    주요 재료 : 마라, 건두부, 목이버섯
    비슷한 음식 : 마라탕
    """
    },
]

# DB에서 가져올수도있고 직접 쓸수도있으므로 형식을 정해줘야함
example_tamplate = """
    Human : {question},
    AI : {answer}
"""
# example_prompt = PromptTemplate.from_template(example_tamplate) # 답변에 대한 템플릿을 지정해줄 수 있음
example_prompt = PromptTemplate.from_template("Human : {question}\nAI:{answer}") # 답변 형식을 지정해줌
prompt = FewShotPromptTemplate(
    example_prompt =example_prompt, 
    examples = examples,
    suffix  = "Human : {food}를 아니" ,# 뒤에 나올 고정말 
    input_variables = ['food'] # 유효성 검사 해줌 - prompt.format(food = "김치찌개") 에 food 가 없으면 에러남

)
# 랭체인이 알아서 예제 리스트들을 형식화할것

prompt.format(food = "파전")

chain = prompt | chat 

chain.invoke({"food":"파전"})


[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "food": "파전"
}
[chain/start] [1:chain:RunnableSequence > 2:prompt:FewShotPromptTemplate] Entering Prompt run with input:
{
  "food": "파전"
}
[chain/end] [1:chain:RunnableSequence > 2:prompt:FewShotPromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "base",
    "StringPromptValue"
  ],
  "kwargs": {
    "text": "Human : 된장찌개 설명해줘\nAI:\n    나라 : 한국\n    주요 재료 : 된장, 두부, 호박\n    비슷한 음식 : 청국장\n    \n\nHuman : 김치찌개\nAI:\n    나라 : 한국\n    주요 재료 : 김치, 돼지고기, 양파 \n    비슷한 음식 : 고추장찌개\n    \n\nHuman : 훠거 설명해줘\nAI:\n    나라 : 중국\n    주요 재료 : 마라, 건두부, 목이버섯\n    비슷한 음식 : 마라탕\n    \n\nHuman : 파전를 아니"
  }
}
[llm/start] [1:chain:RunnableSequence > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Human : 된장찌개 설명해줘\nAI:\n    나라 : 한국\n    주요 재료 : 된장, 두부, 호박\n    비슷한 음식 : 청국장\n    \n\nHuman : 김치찌개\nAI:\n    나라 : 한국\n    주요 재료 

AIMessageChunk(content='AI:\n    나라 : 한국\n    주요 재료 : 부침가루, 파, 해물 (오징어, 새우 등)\n    비슷한 음식 : 김치전')

In [30]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import (
    FewShotPromptTemplate,
    FewShotChatMessagePromptTemplate,
)


model = "gpt-4o-mini"
chat = ChatOpenAI(
    model_name=model,
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


example = [
    {
        "food": "된장찌개",
        "answer": """
    나라 : 한국
    주요 재료 : 된장, 두부, 호박
    비슷한 음식 : 청국장
    """,
    },
    {
        "food": "김치찌개",
        "answer": """
    나라 : 한국
    주요 재료 : 김치, 돼지고기, 양파 
    비슷한 음식 : 고추장찌개
    """,
    },
    {
        "food": "훠거",
        "answer": """
    나라 : 중국
    주요 재료 : 마라, 건두부, 목이버섯
    비슷한 음식 : 마라탕
    """,
    },
]

# 형식을 지정해줌
example_prompts = ChatPromptTemplate.from_messages(
    [("human", "{food}에 대해 설명해줘"), ("ai", "{answer}")]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompts,
    examples=example
)



final_prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 요리전문가야, 답변은 이전과 같은 형식으로 짧게 해줘"),
    example_prompt,  # 예제 메시지를 포함
    ("human", "{food}에 대해 설명해줘")  # 새로운 질문
])

chain = final_prompt | chat

chain.invoke({"food": "파전"})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "food": "파전"
}
[chain/start] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "food": "파전"
}
[chain/end] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "chat",
    "ChatPromptValue"
  ],
  "kwargs": {
    "messages": [
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain",
          "schema",
          "messages",
          "SystemMessage"
        ],
        "kwargs": {
          "content": "너는 요리전문가야, 답변은 이전과 같은 형식으로 짧게 해줘",
          "additional_kwargs": {}
        }
      },
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain",
          "schema",
          "messages",
          "HumanMessage"
        ],
        "kwargs": {
          "content": "된장찌개에 대해 설명해줘

AIMessageChunk(content='\n    나라 : 한국\n    주요 재료 : 부침가루, 파, 해물\n    비슷한 음식 : 전 (전통 전)\n    ')

### 4.3 LengthBasedExampleSelector 
- 동적으로 예제를 선택할 수 있게 하는 것
- 예제를 형식화할 수 있고 , 예제의 양이 얼마나 되는지 알수있음
- 설정해놓은 세팅값에 따라 prompt 양을 결정

In [31]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

# "아래 코드에서 food가 없다는 에러가 나는 이유와 코드를 수정해줘줘"

model = "gpt-4o-mini"
chat = ChatOpenAI(
    model_name=model,
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


example = [{
    "question":"된장찌개 설명해줘",
    "answer" : """
    나라 : 한국
    주요 재료 : 된장, 두부, 호박
    비슷한 음식 : 청국장
    """
    },{
    "question":"김치찌개",
    "answer" : """
    나라 : 한국
    주요 재료 : 김치, 돼지고기, 양파 
    비슷한 음식 : 고추장찌개
    """
    },
    {
    "question":"훠거 설명해줘",
    "answer" : """
    나라 : 중국
    주요 재료 : 마라, 건두부, 목이버섯
    비슷한 음식 : 마라탕
    """
    },
]


example_prompts = PromptTemplate.from_template("Human : {question}\nAI:{answer}") # 답변 형식을 지정해줌


example_selector = LengthBasedExampleSelector(
    examples=example,
    example_prompt=example_prompts,
    max_length=50 # 예제의 양을 얼마나 해줄지 설정가능 (예제가 너무 많으면 랭체인이 알아서 줄여줌) / 숫자를 바꿔서 테스트해보기
)

prompt = FewShotPromptTemplate(
    example_prompt =example_prompts, 
    example_selector = example_selector, # 이전에는 예제를 다 넣어줬지만 이번에는 예제를 선택해줄 수 있음, 원래 exmple을 넣었었음
    suffix  = "Human : {food} 설명해줘" ,# 뒤에 나올 고정말 
    input_variables = ['food'] # 유효성 검사 해줌 - prompt.format(food = "김치찌개") 에 food 가 없으면 에러남

)

prompt.format(food = "굴보쌈")

chain = prompt | chat # t사용자의 질문이 LLM 에 전달되어도 예제들이 허용되는 양을 지키면서 LLM에 전달됨




- 4.3.2 랜덤 예제 선택
    - 나만의 exampel selector를 만들어서 사용하는 방법(원하는대로 예제를 선택할 수 있음)
    - 유저의 로그인 여부/유저 사용 언어에 따라 example을 선택할 수 있음
    - random example selector 구현

In [32]:
# 랜덤으로 예제를 선택하게 하는 방식

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector


model = "gpt-4o-mini"
chat = ChatOpenAI(
    model_name=model,
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

examples = [{
    "question":"된장찌개 설명해줘",
    "answer" : """
    나라 : 한국
    주요 재료 : 된장, 두부, 호박
    비슷한 음식 : 청국장
    """
    },{
    "question":"김치찌개 설명해줘",
    "answer" : """
    나라 : 한국
    주요 재료 : 김치, 돼지고기, 양파 
    비슷한 음식 : 고추장찌개
    """
    },
    {
    "question":"훠거 설명해줘",
    "answer" : """
    나라 : 중국
    주요 재료 : 마라, 건두부, 목이버섯
    비슷한 음식 : 마라탕
    """
    },
]

class RandomExampleSelector(BaseExampleSelector): # BaseExampleSelector를 상속받아서 구현
    
    def __init__(self, examples):
        self.examples = examples
        
    # def add_example(self,example: Dict[str, str]) -> Any:
    #     self.examples.append(example)   
    
    def add_example(self, example): # 동적으로 예제를 추가할 수 있도록 함
        self.examples.append(example) # example은 list 형태 
    
    def select_examples(self, input_variables): # input_variables는 사용자가 입력한 변수들
        # example의 예제 중 하나를 리턴해주도록 함
        # 아래 예시는 예제 선택지를 고르는 매우~~ 간단한 예제 -> 심화해서 작성해보기기
        from random import choice
        return [choice(self.examples)] # 무작위에서 하나만 선택
        

RandomExampleSelector
example_prompts = PromptTemplate.from_template("Human : {question} \n AI:{answer}") # 답변 형식을 지정해줌

# TypeError: Can't instantiate abstract class RandomExampleSelector with abstract method add_example : add _example 메소드를 구현해줘야함
example_selector = RandomExampleSelector(
    examples=examples,
)
prompt = FewShotPromptTemplate(
    example_prompt =example_prompts, 
    example_selector = example_selector, # 이전에는 예제를 다 넣어줬지만 이번에는 예제를 선택해줄 수 있음, 원래 exmple을 넣었었음
    suffix  = "Human : {food} 설명해줘" ,# 뒤에 나올 고정말 
    input_variables = ['food'] 

)

prompt.format(food = "굴보쌈")

# chain = prompt | chat # t사용자의 질문이 LLM 에 전달되어도 예제들이 허용되는 양을 지키면서 LLM에 전달됨



'Human : 김치찌개 설명해줘 \n AI:\n    나라 : 한국\n    주요 재료 : 김치, 돼지고기, 양파 \n    비슷한 음식 : 고추장찌개\n    \n\nHuman : 굴보쌈 설명해줘'

#### Serialization and Composition
- 프롬프트를 저장해서 다른 사람과 공유하며 가져다 쓸 수 있도록 하고싶은 경우 활용 (load_prompt)
- 두가지 타입의 prompt를 만들거임(json, yml)

In [33]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate

from langchain.prompts import load_prompt

# prompt = load_prompt("./prompt.json")
prompt = load_prompt("./prompt.yaml")



model = "gpt-4o-mini"
chat = ChatOpenAI(
    model_name=model,
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

prompt.format(country="France") # prompt에 변수넣고 템플릿을 볼수있음


'What is the capital of France?'

- 많은 프롬프트들의 memory 등을 다 모으는 방법
   - 많은 프롬프트가 있을때 유용함
   - 하나로 다 합쳐야하기 때문

In [34]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler

from langchain.prompts.pipeline import PipelinePromptTemplate

# prompt = load_prompt("./prompt.json")
prompt = load_prompt("./prompt.yaml")



model = "gpt-4o-mini"
chat = ChatOpenAI(
    model_name=model,
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# 4개의 프롬프트를 만들기

# 소개용 프롬프트
intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

# 예제용 프롬프트
example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)


start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

# 마지막으로 합쳐주는 프롬프트트
final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)
prompts = [
    ("intro",intro), # key값은 final 의 변수명과 같아야함
    ("example",example),
    ("start",start)
]

full_prompt = PipelinePromptTemplate(final_prompt=final, pipeline_prompts=prompts)
a=full_prompt.format(
    character ="Pirate",
    example_question="what is your location?", # 예시 질문문
    example_answer="Arrrg! That is a secret!", # 예시 답변
    question="What is your fav food?" # 실제 질문

)
print(a)
# full_prompt





    
    You are a role playing assistant.
    And you are impersonating a Pirate

                                     
    
    This is an example of how you talk:

    Human: what is your location?
    You: Arrrg! That is a secret!

                              
    
    Start now!

    Human: What is your fav food?
    You:




In [35]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler

from langchain.prompts.pipeline import PipelinePromptTemplate

# prompt = load_prompt("./prompt.json")
prompt = load_prompt("./prompt.yaml")



model = "gpt-4o-mini"
chat = ChatOpenAI(
    model_name=model,
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# 4개의 프롬프트를 만들기

# 소개용 프롬프트
intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

# 예제용 프롬프트
example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)


start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

# 마지막으로 합쳐주는 프롬프트트
final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)
prompts = [
    ("intro",intro), # key값은 final 의 변수명과 같아야함
    ("example",example),
    ("start",start)
]

full_prompt = PipelinePromptTemplate(final_prompt=final, pipeline_prompts=prompts)

# chain으로 활용하기
chain = full_prompt | chat
chain.invoke({
    "character" :"Pirate",
    "example_question":"what is your location?", # 예시 질문문
    "example_answer":"Arrrg! That is a secret!", # 예시 답변
    "question":"What is your fav food?" # 실제 질문
    
})


[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "character": "Pirate",
  "example_question": "what is your location?",
  "example_answer": "Arrrg! That is a secret!",
  "question": "What is your fav food?"
}
[chain/start] [1:chain:RunnableSequence > 2:prompt:PipelinePromptTemplate] Entering Prompt run with input:
{
  "character": "Pirate",
  "example_question": "what is your location?",
  "example_answer": "Arrrg! That is a secret!",
  "question": "What is your fav food?"
}
[chain/end] [1:chain:RunnableSequence > 2:prompt:PipelinePromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "base",
    "StringPromptValue"
  ],
  "kwargs": {
    "text": "\n    \n    You are a role playing assistant.\n    And you are impersonating a Pirate\n\n                                     \n    \n    This is an example of how you talk:\n\n    Human: what is your location?\n    You: Arrrg! That is a

AIMessageChunk(content="Arrrg! Me favorite grub be a hearty bowl o' grog and a side o' salted fish! Nothin' like a good feast after a long day o' plunderin' the high seas! What be yer favorite dish, matey?")

#### 4.5 Caching
- I/O 모듈 
- streamlit에 대해 배우고 랭체인과 streamlit을 써보기



In [36]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache

# 챗봇이 있고 그 채팅봇이 항상 똑같은 질문을 받는다면, 답변을 계속 만들지 않고, 이미 답변한 답을 캐싱하여 이용하고 저장하여 재사용할 수 있도록 함
# 비용을 아끼는데 효율적임 . 같은 질문을 하면 같은 답변이 나올 것임

set_llm_cache(InMemoryCache()) # 캐시를 사용하겠다고 선언 /모든 response를 캐싱하여 메모리에 저장함


model = "gpt-4o-mini"
chat = ChatOpenAI(
    model_name=model,
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

# 처음 질문때와 두번째 같은 질문할때 시간이 얼마나 걸리는 지 확인

# chat.predict("이탈리아 파스타는 어떻게 만드나요?") # 처음 질문 : 11.3초

In [37]:
chat.predict("이탈리아 파스타는 어떻게 만드나요?") # 즉시답변함 / 캐싱을 통해 chat을 이용하지 않고 바로 답변함/ 똑같은 질문을 계속 받아도 답변을 재사용하여 비용을 아낄 수 있음

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: 이탈리아 파스타는 어떻게 만드나요?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [12.70s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "이탈리아 파스타는 다양한 종류와 조리법이 있지만, 기본적인 파스타를 만드는 방법을 소개하겠습니다. 여기서는 밀가루와 계란을 사용한 전통적인 이탈리아식 파스타 반죽을 만드는 방법을 설명할게요.\n\n### 재료\n- 밀가루 (00형 또는 일반 밀가루) 100g\n- 계란 1개\n- 소금 약간\n- 올리브 오일 (선택 사항)\n\n### 만드는 방법\n\n1. **반죽 준비하기**:\n   - 깨끗한 작업대에 밀가루를 쌓아 원형으로 만들고, 가운데에 계란을 깨서 넣습니다.\n   - 소금을 약간 넣고, 원형의 밀가루 가장자리를 계란으로 덮어줍니다.\n\n2. **반죽하기**:\n   - 포크를 사용해 계란을 휘저어 밀가루와 섞어줍니다. 점차 밀가루를 계란에 섞어가며 반죽을 만듭니다.\n   - 반죽이 어느 정도 뭉쳐지면 손으로 반죽을 치대어 부드럽고 탄력 있는 반죽이 될 때까지 약 10분 정도 반죽합니다.\n\n3. **휴지시키기**:\n   - 반죽을 랩으로 싸서 30분 정도 실온에서 휴지시킵니다. 이 과정은 글루텐이 안정화되어 반죽이 더 부드러워지게 합니다.\n\n4. **파스타 모양 만들기**:\n   - 휴지시킨 반죽을 밀대로 얇게 밀어줍니다. 원하는 두께로 밀어준 후, 원하는 형태로 잘라줍니다. (예: 태그리넬리, 파르팔레 등)\n   - 또는 파스타 기계를 사용해도 좋습니다.\n\n5. **파스타 삶기**:\n   - 큰 냄비에 물을 끓이고 소금을 넣습니다. \n   - 잘라놓은 파스타를 넣고 2-4분 정도 삶습니다. (파스타의 두께에 따라

'이탈리아 파스타는 다양한 종류와 조리법이 있지만, 기본적인 파스타를 만드는 방법을 소개하겠습니다. 여기서는 밀가루와 계란을 사용한 전통적인 이탈리아식 파스타 반죽을 만드는 방법을 설명할게요.\n\n### 재료\n- 밀가루 (00형 또는 일반 밀가루) 100g\n- 계란 1개\n- 소금 약간\n- 올리브 오일 (선택 사항)\n\n### 만드는 방법\n\n1. **반죽 준비하기**:\n   - 깨끗한 작업대에 밀가루를 쌓아 원형으로 만들고, 가운데에 계란을 깨서 넣습니다.\n   - 소금을 약간 넣고, 원형의 밀가루 가장자리를 계란으로 덮어줍니다.\n\n2. **반죽하기**:\n   - 포크를 사용해 계란을 휘저어 밀가루와 섞어줍니다. 점차 밀가루를 계란에 섞어가며 반죽을 만듭니다.\n   - 반죽이 어느 정도 뭉쳐지면 손으로 반죽을 치대어 부드럽고 탄력 있는 반죽이 될 때까지 약 10분 정도 반죽합니다.\n\n3. **휴지시키기**:\n   - 반죽을 랩으로 싸서 30분 정도 실온에서 휴지시킵니다. 이 과정은 글루텐이 안정화되어 반죽이 더 부드러워지게 합니다.\n\n4. **파스타 모양 만들기**:\n   - 휴지시킨 반죽을 밀대로 얇게 밀어줍니다. 원하는 두께로 밀어준 후, 원하는 형태로 잘라줍니다. (예: 태그리넬리, 파르팔레 등)\n   - 또는 파스타 기계를 사용해도 좋습니다.\n\n5. **파스타 삶기**:\n   - 큰 냄비에 물을 끓이고 소금을 넣습니다. \n   - 잘라놓은 파스타를 넣고 2-4분 정도 삶습니다. (파스타의 두께에 따라 다를 수 있습니다.)\n   - 삶은 후에는 체에 받쳐 물기를 빼고, 원하는 소스와 함께 섞어줍니다.\n\n### 소스 추천\n- **토마토 소스**: 신선한 토마토, 마늘, 올리브 오일, 바질로 만든 소스.\n- **알프레도 소스**: 크림, 버터, 파마산 치즈로 만든 부드러운 소스.\n- **페스토 소스**: 바질, 올리브 오일, 파마산 치즈, 잣으로 만든 소스.\n\n이렇게 만든 파스타는 신선하고 맛있습니다.

In [38]:
# set_debug 이용해보기

from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache

# 챗봇이 있고 그 채팅봇이 항상 똑같은 질문을 받는다면, 답변을 계속 만들지 않고, 이미 답변한 답을 캐싱하여 이용하고 저장하여 재사용할 수 있도록 함
# 비용을 아끼는데 효율적임 . 같은 질문을 하면 같은 답변이 나올 것임

set_llm_cache(InMemoryCache()) # 캐시를 사용하겠다고 선언 /모든 response를 캐싱하여 메모리에 저장함
set_debug(True) # 디버깅을 위한 옵션 / 디버깅을 위한 로그를 출력함

model = "gpt-4o-mini"
chat = ChatOpenAI(
    model_name=model,
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

# 처음 질문때와 두번째 같은 질문할때 시간이 얼마나 걸리는 지 확인

chat.predict("이탈리아 파스타는 어떻게 만드나요?") # 처음 질문 : 11.3초

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: 이탈리아 파스타는 어떻게 만드나요?"
  ]
}
[chain/error] [1:llm:ChatOpenAI] [12.65s] Llm run errored with error:
"KeyboardInterrupt()"


KeyboardInterrupt: 

In [24]:
chat.predict("이탈리아 파스타는 어떻게 만드나요?") # 캐싱됨됨

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: 이탈리아 파스타는 어떻게 만드나요?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "이탈리아 파스타를 만드는 방법은 여러 가지가 있지만, 기본적인 파스타 반죽을 만드는 방법을 소개해드릴게요. \n\n### 기본 파스타 반죽 재료\n- 밀가루 (00형 또는 일반 밀가루) 400g\n- 계란 4개\n- 소금 1작은술\n- 올리브 오일 (선택사항) 1큰술\n\n### 만드는 방법\n\n1. **밀가루 준비**: 깨끗한 작업대나 큰 볼에 밀가루를 쌓아 원형으로 만들고, 가운데에 우물을 만듭니다.\n\n2. **계란 추가**: 우물 중앙에 계란을 깨고 소금과 올리브 오일을 추가합니다.\n\n3. **반죽하기**: 포크를 사용해 계란을 휘저어가며 밀가루를 조금씩 섞어줍니다. 반죽이 뭉쳐지기 시작하면 손으로 반죽을 치대기 시작합니다.\n\n4. **반죽 치대기**: 반죽이 부드럽고 탄력 있게 될 때까지 약 10분 정도 치대줍니다. 필요에 따라 밀가루를 조금 더 추가할 수 있습니다.\n\n5. **휴지**: 반죽을 랩으로 싸서 최소 30분간 실온에서 휴지시킵니다. 이 과정은 글루텐이 안정화되어 반죽이 더 부드러워지게 합니다.\n\n6. **밀어내기**: 휴지 후 반죽을 원하는 두께로 밀어냅니다. 파스타 기계가 있다면 기계를 사용해도 좋습니다.\n\n7. **자르기**: 원하는 형태로 자릅니다. 예를 들어, 파르팔레(나비 모양), 태그리올리니(얇은 면), 라자냐(넓은 면) 등으로 자를 수 있습니다.\n\n8. **삶기**: 끓는 소금물에 파스타를 넣고 2-4분 정도 삶습니다. (생 파스타는 일반적으로 빠르게 익습니다.)\n\n9. **소스와 함께 제공**:

'이탈리아 파스타를 만드는 방법은 여러 가지가 있지만, 기본적인 파스타 반죽을 만드는 방법을 소개해드릴게요. \n\n### 기본 파스타 반죽 재료\n- 밀가루 (00형 또는 일반 밀가루) 400g\n- 계란 4개\n- 소금 1작은술\n- 올리브 오일 (선택사항) 1큰술\n\n### 만드는 방법\n\n1. **밀가루 준비**: 깨끗한 작업대나 큰 볼에 밀가루를 쌓아 원형으로 만들고, 가운데에 우물을 만듭니다.\n\n2. **계란 추가**: 우물 중앙에 계란을 깨고 소금과 올리브 오일을 추가합니다.\n\n3. **반죽하기**: 포크를 사용해 계란을 휘저어가며 밀가루를 조금씩 섞어줍니다. 반죽이 뭉쳐지기 시작하면 손으로 반죽을 치대기 시작합니다.\n\n4. **반죽 치대기**: 반죽이 부드럽고 탄력 있게 될 때까지 약 10분 정도 치대줍니다. 필요에 따라 밀가루를 조금 더 추가할 수 있습니다.\n\n5. **휴지**: 반죽을 랩으로 싸서 최소 30분간 실온에서 휴지시킵니다. 이 과정은 글루텐이 안정화되어 반죽이 더 부드러워지게 합니다.\n\n6. **밀어내기**: 휴지 후 반죽을 원하는 두께로 밀어냅니다. 파스타 기계가 있다면 기계를 사용해도 좋습니다.\n\n7. **자르기**: 원하는 형태로 자릅니다. 예를 들어, 파르팔레(나비 모양), 태그리올리니(얇은 면), 라자냐(넓은 면) 등으로 자를 수 있습니다.\n\n8. **삶기**: 끓는 소금물에 파스타를 넣고 2-4분 정도 삶습니다. (생 파스타는 일반적으로 빠르게 익습니다.)\n\n9. **소스와 함께 제공**: 삶은 파스타를 원하는 소스와 함께 섞어 서빙합니다. 토마토 소스, 알프레도 소스, 바질 페스토 등 다양한 소스를 사용할 수 있습니다.\n\n이렇게 기본적인 이탈리아 파스타를 만들 수 있습니다. 다양한 재료와 소스를 활용해 나만의 스타일로 즐겨보세요!'

In [ ]:
# SQLiteCache : 결과를 DB에 캐싱하고싶을때 / 메모리에 캐싱하면 서버를 껐다 켜면 사라지므로 DB에 저장함


from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

# 챗봇이 있고 그 채팅봇이 항상 똑같은 질문을 받는다면, 답변을 계속 만들지 않고, 이미 답변한 답을 캐싱하여 이용하고 저장하여 재사용할 수 있도록 함
# 비용을 아끼는데 효율적임 . 같은 질문을 하면 같은 답변이 나올 것임

set_llm_cache(SQLiteCache("cache.db")) # 이름은 뭐든 상관없음(폴더를 보면 자동으로 sqlit db가 생긴걸 확인할 숭 ㅣㅆ음)/ DB에 캐싱하는데 .langchain.db를 사용하게 됨

model = "gpt-4o-mini"
chat = ChatOpenAI(
    model_name=model,
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

# 처음 질문때와 두번째 같은 질문할때 시간이 얼마나 걸리는 지 확인

chat.predict("김치찌개 어떻게 만드나요?") # 처음 질문 : 약 8초초

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: 김치찌개 어떻게 만드나요?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [7.85s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "김치찌개는 한국의 전통적인 찌개로, 간단하면서도 맛있는 요리입니다. 아래는 기본적인 김치찌개 레시피입니다.\n\n### 재료\n- 잘 익은 김치 2컵\n- 돼지고기 (목살 또는 삼겹살) 200g\n- 두부 1/2모\n- 대파 1대\n- 양파 1개\n- 마늘 3~4쪽\n- 고춧가루 1~2큰술 (취향에 따라 조절)\n- 국간장 1큰술\n- 소금, 후추 약간\n- 물 4컵\n\n### 조리 방법\n1. **재료 손질**: \n   - 김치는 먹기 좋은 크기로 자릅니다.\n   - 돼지고기는 한 입 크기로 썰고, 두부는 큐브 모양으로 자릅니다.\n   - 대파는 어슷하게 썰고, 양파는 채 썰고, 마늘은 다집니다.\n\n2. **고기 볶기**: \n   - 냄비에 돼지고기를 넣고 중불에서 볶아 기름이 나올 때까지 익힙니다.\n\n3. **김치 추가**: \n   - 볶은 고기에 김치를 넣고 함께 볶아 김치가 약간 익을 때까지 볶습니다.\n\n4. **물 붓기**: \n   - 볶은 재료에 물을 붓고 끓입니다.\n\n5. **양념하기**: \n   - 끓기 시작하면 고춧가루, 국간장, 다진 마늘을 넣고 잘 섞습니다.\n\n6. **재료 넣기**: \n   - 양파와 두부를 넣고 중불에서 10~15분 정도 끓입니다. 이때 소금과 후추로 간을 맞춥니다.\n\n7. **마무리**: \n   - 마지막으로 대파를 넣고 2~3분 더 끓인 후 불을 끕니다.\n\n8. **서빙**: \n   - 뜨거운 김치찌개를 그릇에 담아 밥과 함께 즐기세요.\n\n맛있게 드세요!",
     

'김치찌개는 한국의 전통적인 찌개로, 간단하면서도 맛있는 요리입니다. 아래는 기본적인 김치찌개 레시피입니다.\n\n### 재료\n- 잘 익은 김치 2컵\n- 돼지고기 (목살 또는 삼겹살) 200g\n- 두부 1/2모\n- 대파 1대\n- 양파 1개\n- 마늘 3~4쪽\n- 고춧가루 1~2큰술 (취향에 따라 조절)\n- 국간장 1큰술\n- 소금, 후추 약간\n- 물 4컵\n\n### 조리 방법\n1. **재료 손질**: \n   - 김치는 먹기 좋은 크기로 자릅니다.\n   - 돼지고기는 한 입 크기로 썰고, 두부는 큐브 모양으로 자릅니다.\n   - 대파는 어슷하게 썰고, 양파는 채 썰고, 마늘은 다집니다.\n\n2. **고기 볶기**: \n   - 냄비에 돼지고기를 넣고 중불에서 볶아 기름이 나올 때까지 익힙니다.\n\n3. **김치 추가**: \n   - 볶은 고기에 김치를 넣고 함께 볶아 김치가 약간 익을 때까지 볶습니다.\n\n4. **물 붓기**: \n   - 볶은 재료에 물을 붓고 끓입니다.\n\n5. **양념하기**: \n   - 끓기 시작하면 고춧가루, 국간장, 다진 마늘을 넣고 잘 섞습니다.\n\n6. **재료 넣기**: \n   - 양파와 두부를 넣고 중불에서 10~15분 정도 끓입니다. 이때 소금과 후추로 간을 맞춥니다.\n\n7. **마무리**: \n   - 마지막으로 대파를 넣고 2~3분 더 끓인 후 불을 끕니다.\n\n8. **서빙**: \n   - 뜨거운 김치찌개를 그릇에 담아 밥과 함께 즐기세요.\n\n맛있게 드세요!'

In [27]:
chat.predict("김치찌개 어떻게 만드나요?") 

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: 김치찌개 어떻게 만드나요?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "김치찌개는 한국의 전통적인 찌개로, 간단하면서도 맛있는 요리입니다. 아래는 기본적인 김치찌개 레시피입니다.\n\n### 재료\n- 잘 익은 김치 2컵\n- 돼지고기 (목살 또는 삼겹살) 200g\n- 두부 1/2모\n- 대파 1대\n- 양파 1개\n- 마늘 3~4쪽\n- 고춧가루 1~2큰술 (취향에 따라 조절)\n- 국간장 1큰술\n- 소금, 후추 약간\n- 물 4컵\n\n### 조리 방법\n1. **재료 손질**: \n   - 김치는 먹기 좋은 크기로 자릅니다.\n   - 돼지고기는 한 입 크기로 썰고, 두부는 큐브 모양으로 자릅니다.\n   - 대파는 어슷하게 썰고, 양파는 채 썰고, 마늘은 다집니다.\n\n2. **고기 볶기**: \n   - 냄비에 돼지고기를 넣고 중불에서 볶아 기름이 나올 때까지 익힙니다.\n\n3. **김치 추가**: \n   - 볶은 고기에 김치를 넣고 함께 볶아 김치가 약간 익을 때까지 볶습니다.\n\n4. **물 붓기**: \n   - 볶은 재료에 물을 붓고 끓입니다.\n\n5. **양념하기**: \n   - 끓기 시작하면 고춧가루, 국간장, 다진 마늘을 넣고 잘 섞습니다.\n\n6. **재료 넣기**: \n   - 양파와 두부를 넣고 중불에서 10~15분 정도 끓입니다. 이때 소금과 후추로 간을 맞춥니다.\n\n7. **마무리**: \n   - 마지막으로 대파를 넣고 2~3분 더 끓인 후 불을 끕니다.\n\n8. **서빙**: \n   - 뜨거운 김치찌개를 그릇에 담아 밥과 함께 즐기세요.\n\n맛있게 드세요!",
        "g

'김치찌개는 한국의 전통적인 찌개로, 간단하면서도 맛있는 요리입니다. 아래는 기본적인 김치찌개 레시피입니다.\n\n### 재료\n- 잘 익은 김치 2컵\n- 돼지고기 (목살 또는 삼겹살) 200g\n- 두부 1/2모\n- 대파 1대\n- 양파 1개\n- 마늘 3~4쪽\n- 고춧가루 1~2큰술 (취향에 따라 조절)\n- 국간장 1큰술\n- 소금, 후추 약간\n- 물 4컵\n\n### 조리 방법\n1. **재료 손질**: \n   - 김치는 먹기 좋은 크기로 자릅니다.\n   - 돼지고기는 한 입 크기로 썰고, 두부는 큐브 모양으로 자릅니다.\n   - 대파는 어슷하게 썰고, 양파는 채 썰고, 마늘은 다집니다.\n\n2. **고기 볶기**: \n   - 냄비에 돼지고기를 넣고 중불에서 볶아 기름이 나올 때까지 익힙니다.\n\n3. **김치 추가**: \n   - 볶은 고기에 김치를 넣고 함께 볶아 김치가 약간 익을 때까지 볶습니다.\n\n4. **물 붓기**: \n   - 볶은 재료에 물을 붓고 끓입니다.\n\n5. **양념하기**: \n   - 끓기 시작하면 고춧가루, 국간장, 다진 마늘을 넣고 잘 섞습니다.\n\n6. **재료 넣기**: \n   - 양파와 두부를 넣고 중불에서 10~15분 정도 끓입니다. 이때 소금과 후추로 간을 맞춥니다.\n\n7. **마무리**: \n   - 마지막으로 대파를 넣고 2~3분 더 끓인 후 불을 끕니다.\n\n8. **서빙**: \n   - 뜨거운 김치찌개를 그릇에 담아 밥과 함께 즐기세요.\n\n맛있게 드세요!'

참고 링크 : https://python.langchain.com/docs/integrations
- langchain 문서로 들어가서 integrations 로 들어가면 모든 써드파티 제공업체를 확인할 수 있음
- 채팅 모델용 llm을 지원하고, 다른 종류의 loader도 지원함
  - text embbeding, retrivate, 캐싱 지원하는 다른 방법 등등

- 캐싱관련
  - https://python.langchain.com/docs/integrations/llm_caching/
  - 다양한 캐싱방법을 지원하므로, 만약 SQLite를 사용하고 싶지 않은 경우, 다른 종류의 캐싱 방법을 사용하거나 직접 개발할 수 있음

#### 4.6 serialization
- openAI 모델 사용시 지불하는 비용 계산하는 방법
- 두번째는 모델을 어떻게 저장하고 불러오는지 강의의

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

set_debug(False)
model = "gpt-4o-mini"
chat = ChatOpenAI(
    model_name=model,
    temperature=0.1,
)

with get_openai_callback() as usage:
    a= chat.predict("zookeeper는 hbase가 없어도 hadoop hdfs만 구축한 환경에도 필요한거야?") # 콜백을 사용하여 예측함
    b= chat.predict("hadoop에 대해 name node, datanode 구성시 zookeeper가 꼭 필요한가?")
    print(a,b,"\n")
    print(usage) # 사용량을 출력함 / 그 외, usate.cost, usage.tokens, usage.successful_requests 등을 출력할 수 있음
'''
Tokens Used: 495
	Prompt Tokens: 54
	Completion Tokens: 441
Successful Requests: 2 # 두번 호출한 횟수 
Total Cost (USD): $0.0 # 사용금액
'''

Zookeeper는 HBase와 같은 분산 시스템에서 주로 사용되지만, Hadoop HDFS 환경에서도 필요할 수 있습니다. HDFS 자체는 Zookeeper 없이도 작동하지만, Zookeeper는 다음과 같은 경우에 유용할 수 있습니다:

1. **클러스터 관리**: Zookeeper는 클러스터의 상태를 모니터링하고, 노드의 추가 및 제거를 관리하는 데 도움을 줄 수 있습니다.

2. **메타데이터 관리**: HDFS의 메타데이터를 관리하는 데 Zookeeper를 사용할 수 있습니다. 예를 들어, Namenode의 장애 조치(failover) 시 Zookeeper가 유용할 수 있습니다.

3. **분산 애플리케이션**: Hadoop 생태계에서 다른 분산 애플리케이션(예: YARN, Kafka 등)을 사용할 경우, Zookeeper가 필요할 수 있습니다.

결론적으로, HDFS만 사용하는 경우 Zookeeper는 필수는 아니지만, 클러스터 관리나 장애 조치와 같은 특정 요구 사항이 있을 경우 유용하게 사용할 수 있습니다. Hadoop의 기본 구성에서 NameNode와 DataNode는 HDFS (Hadoop Distributed File System)의 핵심 구성 요소입니다. NameNode는 메타데이터를 관리하고, DataNode는 실제 데이터를 저장합니다. 

ZooKeeper는 Hadoop의 기본 구성 요소는 아니지만, 특정 상황에서는 유용하게 사용될 수 있습니다. 예를 들어, Hadoop 클러스터의 고가용성을 위해 HA (High Availability) 구성을 설정할 때 ZooKeeper를 사용할 수 있습니다. 이 경우, ZooKeeper는 Active NameNode와 Standby NameNode 간의 상태를 관리하고, 장애 발생 시 자동으로 Active NameNode를 전환하는 역할을 합니다.

따라서, 기본적인 Hadoop HDFS 구성에서는 ZooKeeper가 필수는 아니지만, 고가용성 설정을 원할 경우에는 ZooKeeper가 필요합니다. 

#### chat 모델 저장하고 불러오기

In [43]:
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI


chat = OpenAI(
    model = "gpt-4o-mini",
    temperature = 0.1,
    max_tokens = 450
)
chat.save("model.json") # 모델을 저장함
"""
{
    "model_name": "gpt-4o-mini",
    "temperature": 0.1,
    "max_tokens": 450,
    "top_p": 1,
    "frequency_penalty": 0,
    "presence_penalty": 0,
    "n": 1,
    "request_timeout": null,
    "logit_bias": {},
    "_type": "openai"
}
"""


In [45]:
from langchain.llms.loading import load_llm

chat = load_llm("model.json") # 모델을 불러옴

chat

c:\chloeCode\nomad\fullstackgpt\env3116\Lib\site-packages\langchain\llms\openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\chloeCode\nomad\fullstackgpt\env3116\Lib\site-packages\langchain\llms\openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


OpenAIChat(client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-4o-mini', model_kwargs={'temperature': 0.1, 'max_tokens': 450, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}})